In [20]:
import pyspark

In [21]:
sc

In [22]:
pwd

u'/Users/arnavsomani/Downloads'

In [23]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [24]:

sat = sc.textFile(SAT_FN, use_unicode=False).cache()


list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [25]:
noHeaderRDD = sat.filter(lambda x: not x.startswith('DBN'))
print (sat.first())
print (noHeaderRDD.first())

DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387


In [26]:

def extractScores(partId, records):
    if partId==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[2]!='s': # to filter our bad-quality data
            (dbn,takers,score) = (row[0], int(row[2]), int(row[4]))
            yield (dbn, (score*takers, takers))

mathScores = sat.mapPartitionsWithIndex(extractScores)
mathScores.take(5)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59)),
 ('18K637', (13335, 35))]

In [27]:

schools = sc.textFile(HSD_FN, use_unicode=False).cache()
list(enumerate(schools.first().split(',')))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

In [28]:
def extractBus(partId, list_of_records):
    if partId==0: 
        list_of_records.next() # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[10]!='N/A':
            (dbn, bus) = (row[0], row[10])
            yield (dbn, bus)

busLine = schools.mapPartitionsWithIndex(extractBus)
#busLine.take(7)

In [29]:
bus = busLine.join(mathScores).values().reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: x[0]/x[1])


bus.take(7)

[('Q17, Q25, Q3, Q30, Q31, Q36, Q43, Q46, Q65, Q76', 478),
 ('Q29, Q38, Q53, Q58, Q59, Q60, Q72, Q88', 440),
 ('M14A, M14D, M21, M22, M9', 604),
 ('B13, B14, Q24, Q7, Q8', 418),
 ('Q27, Q30', 545),
 ('M1, M101, M102, M106, M116, M15, M15-SBS, M2, M3', 446),
 ('B24, B32, B39, B44, B44-SBS, B46, B48, B60, B62, Q54, Q59', 395)]

In [30]:
 busMaths= bus.flatMap(lambda row: zip(row[0].split(','),[ row[1]]*len(row[0])) ).collect()

busMaths

[('Q17', 478),
 (' Q25', 478),
 (' Q3', 478),
 (' Q30', 478),
 (' Q31', 478),
 (' Q36', 478),
 (' Q43', 478),
 (' Q46', 478),
 (' Q65', 478),
 (' Q76', 478),
 ('Q29', 440),
 (' Q38', 440),
 (' Q53', 440),
 (' Q58', 440),
 (' Q59', 440),
 (' Q60', 440),
 (' Q72', 440),
 (' Q88', 440),
 ('M14A', 604),
 (' M14D', 604),
 (' M21', 604),
 (' M22', 604),
 (' M9', 604),
 ('B13', 418),
 (' B14', 418),
 (' Q24', 418),
 (' Q7', 418),
 (' Q8', 418),
 ('Q27', 545),
 (' Q30', 545),
 ('M1', 446),
 (' M101', 446),
 (' M102', 446),
 (' M106', 446),
 (' M116', 446),
 (' M15', 446),
 (' M15-SBS', 446),
 (' M2', 446),
 (' M3', 446),
 ('B24', 395),
 (' B32', 395),
 (' B39', 395),
 (' B44', 395),
 (' B44-SBS', 395),
 (' B46', 395),
 (' B48', 395),
 (' B60', 395),
 (' B62', 395),
 (' Q54', 395),
 (' Q59', 395),
 ('B13', 465),
 (' B26', 465),
 (' B38', 465),
 (' B52', 465),
 (' B54', 465),
 (' B60', 465),
 (' Q55', 465),
 (' Q58', 465),
 ('S54', 474),
 (' S57', 474),
 (' S61', 474),
 ('Bx6', 349),
 (' Bx7', 3

In [31]:
def extractSubway(partId, list_of_records):
    if partId==0: 
        list_of_records.next() # skipping the first line
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[11]!='N/A':
            (dbn, sub) = (row[0], row[11])
            yield (dbn, sub)

subLine = schools.mapPartitionsWithIndex(extractSubway)

In [32]:
sub = subLine.join(mathScores).values().reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
   .mapValues(lambda x: x[0]/x[1])

sub.take(5)

[('3 to Harlem - 148 St ; A, B, C, D to 145th St', 474),
 ('B, Q to Church Ave ; F, G to Fort Hamilton Parkway', 456),
 ('1 to 137th St - City College ; A, D to 125th St ; B, C to 135th St', 456),
 ('2, 3 to Central Park North-110th St ; 6 to 103rd St', 446),
 ('1, C, E, F, M to 23rd St ; 2, 3 to 34th St - Penn Station ; A, L to 14th St-8th Ave ; B, D, Q to 34th St-Herald Square ; N, R to 28th St',
  423)]

In [33]:
subMath = sub.flatMap(lambda row: zip(row[0].split(','),[ row[1]]*len(row[0])) ).collect()

subMath


[('3 to Harlem - 148 St ; A', 474),
 (' B', 474),
 (' C', 474),
 (' D to 145th St', 474),
 ('B', 456),
 (' Q to Church Ave ; F', 456),
 (' G to Fort Hamilton Parkway', 456),
 ('1 to 137th St - City College ; A', 456),
 (' D to 125th St ; B', 456),
 (' C to 135th St', 456),
 ('2', 446),
 (' 3 to Central Park North-110th St ; 6 to 103rd St', 446),
 ('1', 423),
 (' C', 423),
 (' E', 423),
 (' F', 423),
 (' M to 23rd St ; 2', 423),
 (' 3 to 34th St - Penn Station ; A', 423),
 (' L to 14th St-8th Ave ; B', 423),
 (' D', 423),
 (' Q to 34th St-Herald Square ; N', 423),
 (' R to 28th St', 423),
 ('M', 449),
 (' R to 46th St', 449),
 ('G to Metropolitan Ave-Lorimer St ; J', 483),
 (' M to Lorimer St ; L to Grand St', 483),
 ('M', 568),
 (' R to 36th St ; N', 568),
 (' Q to 36 Ave-Washington Ave', 568),
 ('A', 419),
 (' S to Beach 98th St-Playland', 419),
 ('B', 496),
 (' Q to Ave M', 496),
 ('F to East Broadway ; J', 423),
 (' M', 423),
 (' Z to Delancey St-Essex St', 423),
 ('1', 426),
 (' R 